In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)

## Cleaning Data

In [2]:
buoy_df = pd.read_csv('/Users/brianjoseph/Desktop/Cod-Squad/data/buoy_data.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
buoy_df = buoy_df[['Time-UTC'] + [col for col in buoy_df.columns if 'M01-' in col or 'N01-' in col]]

In [4]:
buoy_times = pd.DataFrame((buoy_df['Time-UTC'].str.extractall(
    '(?P<year>\d{4})-(?P<month>\d{2})-(?P<date>\d{2}) (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2})')))\
.reset_index()[['year', 'month', 'date', 'hour', 'minute']]

In [5]:
buoy_df = pd.merge(buoy_times, buoy_df.drop('Time-UTC', axis = 1), how = 'left', left_index = True, right_index = True)
buoy_df = buoy_df[buoy_df['year'] != '2019']
buoy_df = buoy_df.drop(buoy_df[buoy_df['M01-Hourly-Visibility_m'] == 'M01-Hourly-Visibility_m'].index)

for col in buoy_df.columns:
    buoy_df[col] = buoy_df[col].astype(float)

In [6]:
TIME_COLS = ['year', 'month', 'date', 'hour', 'minute']
m01_df = buoy_df[TIME_COLS + [col for col in buoy_df.columns if 'M01-' in col]]
n01_df = buoy_df[TIME_COLS + [col for col in buoy_df.columns if 'N01-' in col]]

In [7]:
m01_df.columns = [col.replace('M01-Hourly-', '').lower() for col in m01_df.columns]
n01_df.columns = [col.replace('N01-Hourly-', '').lower() for col in n01_df.columns]

In [8]:
m01_df.to_csv('m01_buoy.csv')
n01_df.to_csv('n01_buoy.csv')